# Query Classifier Tutorial
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial14_Query_Classifier.ipynb)

In this tutorial we introduce the query classifier the goal of introducing this feature was to optimize the overall flow of Haystack pipeline by detecting the nature of user queries. Now, the Haystack can detect primarily three types of queries using both light-weight SKLearn Gradient Boosted classifier or Transformer based more robust classifier. The three categories of queries are as follows:


### 1. Keyword Queries: 
Such queries don't have semantic meaning and merely consist of keywords. For instance these three are the examples of keyword queries.

*   arya stark father
*   jon snow country
*   arya stark younger brothers

### 2. Interrogative Queries: 
In such queries users usually ask a question, regardless of presence of "?" in the query the goal here is to detect the intent of the user whether any question is asked or not in the query. For example:

*   who is the father of arya stark ?
*   which country was jon snow filmed ?
*   who are the younger brothers of arya stark ?

### 3. Declarative Queries: 
Such queries are variation of keyword queries, however, there is semantic relationship between words. Fo example:

*   Arya stark was a daughter of a lord.
*   Jon snow was filmed in a country in UK.
*   Bran was brother of a princess.

In this tutorial, you will learn how the `TransformersQueryClassifier` and `SklearnQueryClassifier` classes can be used to intelligently route your queries, based on the nature of the user query. Also, you can choose between a lightweight Gradients boosted classifier or a transformer based classifier.

Furthermore, there are two types of classifiers you can use out of the box from Haystack.
1. Keyword vs Statement/Question Query Classifier
2. Statement vs Question Query Classifier

As evident from the name the first classifier detects the keywords search queries and semantic statements like sentences/questions. The second classifier differentiates between question based queries and declarative sentences.

### Prepare environment

#### Colab: Enable the GPU runtime
Make sure you enable the GPU runtime to experience decent speed in this tutorial.  
**Runtime -> Change Runtime type -> Hardware accelerator -> GPU**

<img src="https://raw.githubusercontent.com/deepset-ai/haystack/master/docs/_src/img/colab_gpu_runtime.jpg">

These lines are to install Haystack through pip

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install grpcio-tools==1.34.1
!pip install --upgrade git+https://github.com/deepset-ai/haystack.git

# Install  pygraphviz
!apt install libgraphviz-dev
!pip install pygraphviz

# If you run this notebook on Google Colab, you might need to
# restart the runtime after installing haystack.

# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

     |████████████████████████████████| 2.5 MB 7.7 MB/s 
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-dq33uzal
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-dq33uzal
     |████████████████████████████████| 204 kB 8.3 MB/s 
     |████████████████████████████████| 52 kB 984 kB/s 
     |████████████████████████████████| 50 kB 8.1 MB/s 
     |████████████████████████████████| 79 kB 9.4 MB/s 
     |████████████████████████████████| 321 kB 38.6 MB/s 
     |████████████████████████████████| 330 kB 34.8 MB/s 
     |████████████████████████████████| 85 kB 5.7 MB/s 
     |████████████████████████████████| 981 kB 40.1 MB/s 
     |████████████████████████████████| 5.6 MB 45.6 MB/s 
     |████████████████████████████████| 100 kB 11.8 MB/s 
     |████████████████████████████████| 8.4 MB 28.1 MB/s 
     |████████████████████████████████| 344 kB 57.6 MB/s 
     |████████████████████████████████| 56 kB 5.3 MB/s 
     |████████

If running from Colab or a no Docker environment, you will want to start Elasticsearch from source

## Initialization

Here are some core imports

Then let's fetch some data (in this case, pages from the Game of Thrones wiki) and prepare it so that it can
be used indexed into our `DocumentStore`

In [ ]:
from haystack.utils import print_answers, fetch_archive_from_http, convert_files_to_dicts, clean_wiki_text, launch_es
from haystack.pipelines import Pipeline, RootNode
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import ElasticsearchRetriever, DensePassageRetriever, FARMReader, TransformersQueryClassifier, SklearnQueryClassifier

#Download and prepare data - 517 Wikipedia articles for Game of Thrones
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
got_dicts = convert_files_to_dicts(
    dir_path=doc_dir,
    clean_func=clean_wiki_text,
    split_paragraphs=True
)

# Initialize DocumentStore and index documents
launch_es()
document_store = ElasticsearchDocumentStore()
document_store.delete_documents()
document_store.write_documents(got_dicts)

# Initialize Sparse retriever
es_retriever = ElasticsearchRetriever(document_store=document_store)

# Initialize dense retriever
dpr_retriever = DensePassageRetriever(document_store)
document_store.update_embeddings(dpr_retriever, update_existing_embeddings=False)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
08/06/2021 15:42:27 - INFO - haystack.preprocessor.utils -   Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip to `data/article_txt_got`
100%|██████████| 1095120/1095120 [00:00<00:00, 1226455.24B/s]
08/06/2021 15:42:28 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/27_Game_of_Thrones__Season_4__soundtrack_.txt
08/06/2021 15:42:28 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/160_Viserys_Targaryen.txt
08/06/2021 15:42:28 - INFO - haystack.preprocessor.utils -   Converting data/article_txt_got/195_World_of_A_Song_of_Ice_and_Fire.txt
08/06/2021 15:4

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

08/06/2021 15:42:39 - INFO - filelock -   Lock 140420549184400 released on /root/.cache/huggingface/transformers/4ad08b5f983c1384baaf257d8edf51a7a3961fd8c75a1778ac604e3c0b564dd9.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
08/06/2021 15:42:40 - INFO - filelock -   Lock 140420646316304 acquired on /root/.cache/huggingface/transformers/b305bc9085b3d0ce33551c251b75c11b6c6df1d4d51e5d3439d01cf4bb1abc9d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

08/06/2021 15:42:40 - INFO - filelock -   Lock 140420646316304 released on /root/.cache/huggingface/transformers/b305bc9085b3d0ce33551c251b75c11b6c6df1d4d51e5d3439d01cf4bb1abc9d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
08/06/2021 15:42:41 - INFO - filelock -   Lock 140420646314128 acquired on /root/.cache/huggingface/transformers/d5b5f07ee846d5baa7142e121b6ee77d11ac68bd5d4541faab38a1ea76c2954a.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock


...

08/06/2021 15:44:29 - INFO - filelock -   Lock 140417504268880 released on /root/.cache/huggingface/transformers/b87d46371731376b11768b7839b1a5938a4f77d6bd2d9b683f167df0026af432.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
08/06/2021 15:44:30 - INFO - filelock -   Lock 140420403278096 acquired on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock


Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

08/06/2021 15:44:31 - INFO - filelock -   Lock 140420403278096 released on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock
08/06/2021 15:44:31 - INFO - filelock -   Lock 140420403278096 acquired on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

08/06/2021 15:44:31 - INFO - filelock -   Lock 140420403278096 released on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock
08/06/2021 15:44:32 - WARNING - farm.utils -   Failed to log params: Changing param values is not allowed. Param with key='processor' was already logged with value='TextSimilarityProcessor' for run ID='1b048d196ff442d781f6dccac1eaa0a1'. Attempted logging new value 'SquadProcessor'.
08/06/2021 15:44:32 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
08/06/2021 15:44:32 - INFO - farm.utils -   Using device: CUDA 
08/06/2021 15:44:32 - INFO - farm.utils -   Number of GPUs: 1
08/06/2021 15:44:32 - INFO - farm.utils -   Distributed Training: False
08/06/2021 15:44:32 - INFO - farm.utils -   Automatic Mixed Precision: None
08/06/2021 15:44:32 - INFO - farm.infer -   Got ya 2 paralle

## Keyword vs Question/Statement Classifier

The keyword vs question/statement query classifier essentially distinguishes between the keyword queries and statements/questions. So you can intelligently route to different retrieval nodes based on the nature of the query. Using this classifier can potentially yield the following benefits:

*  Getting better search results (e.g. by routing only proper questions to DPR / QA branches and not keyword queries)
*  Less GPU costs (e.g. if 50% of your traffic is only keyword queries you could just use elastic here and save the GPU resources for the other 50% of traffic with semantic queries)

![image](https://user-images.githubusercontent.com/6007894/127831511-f55bad86-4b4f-4b54-9889-7bba37e475c6.png)


Below, we define a `SklQueryClassifier` and show how to use it:

Read more about the trained model and dataset used [here](https://ext-models-haystack.s3.eu-central-1.amazonaws.com/gradboost_query_classifier/readme.txt)

In [ ]:
# Here we build the pipeline
sklearn_keyword_classifier = Pipeline()
sklearn_keyword_classifier.add_node(component=SklearnQueryClassifier(), name="QueryClassifier", inputs=["Query"])
sklearn_keyword_classifier.add_node(component=dpr_retriever, name="DPRRetriever", inputs=["QueryClassifier.output_1"])
sklearn_keyword_classifier.add_node(component=es_retriever, name="ESRetriever", inputs=["QueryClassifier.output_2"])
sklearn_keyword_classifier.add_node(component=reader, name="QAReader", inputs=["ESRetriever", "DPRRetriever"])
sklearn_keyword_classifier.draw("pipeline_classifier.png")


In [ ]:

# Run only the dense retriever on the full sentence query
res_1 = sklearn_keyword_classifier.run(
    query="Who is the father of Arya Stark?"
)
print("DPR Results" + "\n" + "="*15)
print_answers(res_1, details="minimum")

# Run only the sparse retriever on a keyword based query
res_2 = sklearn_keyword_classifier.run(
    query="arya stark father"
)
print("ES Results" + "\n" + "="*15)
print_answers(res_2, details="minimum")


In [ ]:

# Run only the dense retriever on the full sentence query
res_3 = sklearn_keyword_classifier.run(
    query="which country was jon snow filmed ?"
)
print("DPR Results" + "\n" + "="*15)
print_answers(res_3, details="minimum")

# Run only the sparse retriever on a keyword based query
res_4 = sklearn_keyword_classifier.run(
    query="jon snow country"
)
print("ES Results" + "\n" + "="*15)
print_answers(res_4, details="minimum")

In [ ]:
# Run only the dense retriever on the full sentence query
res_5 = sklearn_keyword_classifier.run(
    query="who are the younger brothers of arya stark ?"
)
print("DPR Results" + "\n" + "="*15)
print_answers(res_5, details="minimum")

# Run only the sparse retriever on a keyword based query
res_6 = sklearn_keyword_classifier.run(
    query="arya stark younger brothers"
)
print("ES Results" + "\n" + "="*15)
print_answers(res_6, details="minimum")

## Transformer Keyword vs Question/Statement Classifier

Firstly, it's essential to understand the trade-offs between SkLearn and Transformer query classifiers. The transformer classifier is more accurate than SkLearn classifier however, it requires more memory and most probably GPU for faster inference however the transformer size is roughly `50 MBs`. Whereas, SkLearn is less accurate however is much more faster and doesn't require GPU for inference.

Below, we define a `TransformersQueryClassifier` and show how to use it:

Read more about the trained model and dataset used [here](https://huggingface.co/shahrukhx01/bert-mini-finetune-question-detection)

In [ ]:
# Here we build the pipeline
transformer_keyword_classifier = Pipeline()
transformer_keyword_classifier.add_node(component=TransformersQueryClassifier(), name="QueryClassifier", inputs=["Query"])
transformer_keyword_classifier.add_node(component=dpr_retriever, name="DPRRetriever", inputs=["QueryClassifier.output_1"])
transformer_keyword_classifier.add_node(component=es_retriever, name="ESRetriever", inputs=["QueryClassifier.output_2"])
transformer_keyword_classifier.add_node(component=reader, name="QAReader", inputs=["ESRetriever", "DPRRetriever"])
transformer_keyword_classifier.draw("pipeline_classifier.png")

In [ ]:

# Run only the dense retriever on the full sentence query
res_1 = transformer_keyword_classifier.run(
    query="Who is the father of Arya Stark?"
)
print("DPR Results" + "\n" + "="*15)
print_answers(res_1, details="minimum")

# Run only the sparse retriever on a keyword based query
res_2 = transformer_keyword_classifier.run(
    query="arya stark father"
)
print("ES Results" + "\n" + "="*15)
print_answers(res_2, details="minimum")


In [ ]:

# Run only the dense retriever on the full sentence query
res_3 = transformer_keyword_classifier.run(
    query="which country was jon snow filmed ?"
)
print("DPR Results" + "\n" + "="*15)
print_answers(res_3, details="minimum")

# Run only the sparse retriever on a keyword based query
res_4 = transformer_keyword_classifier.run(
    query="jon snow country"
)
print("ES Results" + "\n" + "="*15)
print_answers(res_4, details="minimum")

In [ ]:
# Run only the dense retriever on the full sentence query
res_5 = transformer_keyword_classifier.run(
    query="who are the younger brothers of arya stark ?"
)
print("DPR Results" + "\n" + "="*15)
print_answers(res_5, details="minimum")

# Run only the sparse retriever on a keyword based query
res_6 = transformer_keyword_classifier.run(
    query="arya stark younger brothers"
)
print("ES Results" + "\n" + "="*15)
print_answers(res_6, details="minimum")

## Question vs Statement Classifier

One possible use case of this classifier could be to route queries after the document retrieval to only send questions to QA reader and in case of declarative sentence, just return the DPR/ES results back to user to enhance user experience and only show answers when user explicitly asks it.

![image](https://user-images.githubusercontent.com/6007894/127864452-f931ea7f-2e62-4f59-85dc-056d56eb9295.png)


Below, we define a `TransformersQueryClassifier` and show how to use it:

Read more about the trained model and dataset used [here](https://huggingface.co/shahrukhx01/question-vs-statement-classifier)

In [ ]:
# Here we build the pipeline
transformer_question_classifier = Pipeline()
transformer_question_classifier.add_node(component=dpr_retriever, name="DPRRetriever", inputs=["Query"])
transformer_question_classifier.add_node(component=TransformersQueryClassifier(model_name_or_path="shahrukhx01/question-vs-statement-classifier"), name="QueryClassifier", inputs=["DPRRetriever"])
transformer_question_classifier.add_node(component=reader, name="QAReader", inputs=["QueryClassifier.output_1"])
transformer_question_classifier.draw("question_classifier.png")

# Run only the QA reader on the question query
res_1 = transformer_question_classifier.run(
    query="Who is the father of Arya Stark?"
)
print("DPR Results" + "\n" + "="*15)
print_answers(res_1, details="minimum")

# Show only DPR results
res_2 = transformer_question_classifier.run(
    query="Arya Stark was the daughter of a Lord."
)
print("ES Results" + "\n" + "="*15)
print_answers(res_2, details="minimum")

## Standalone Query Classifier
Below we run queries classifiers standalone to better understand their outputs on each of the three types of queries

In [ ]:
# Here we create the keyword vs question/statement query classifier
from haystack.pipelines import TransformersQueryClassifier

queries = ["arya stark father","jon snow country",
           "who is the father of arya stark","which country was jon snow filmed?"]

keyword_classifier = TransformersQueryClassifier()

for query in queries:
    result = keyword_classifier.run(query=query)
    if result[1] == "output_1":
        category = "question/statement"
    else:
        category = "keyword"

    print(f"Query: {query}, raw_output: {result}, class: {category}")


Query: arya stark father, raw_output: ({'query': 'arya stark father'}, 'output_2'), class: keyword
Query: jon snow country, raw_output: ({'query': 'jon snow country'}, 'output_2'), class: keyword
Query: who is the father of arya stark, raw_output: ({'query': 'who is the father of arya stark'}, 'output_1'), class: question/statement
Query: which country was jon snow filmed?, raw_output: ({'query': 'which country was jon snow filmed?'}, 'output_1'), class: question/statement


In [ ]:
# Here we create the question vs statement query classifier 
from haystack.pipelines import TransformersQueryClassifier

queries = ["Lord Eddard was the father of Arya Stark.","Jon Snow was filmed in United Kingdom.",
           "who is the father of arya stark?","Which country was jon snow filmed in?"]

question_classifier = TransformersQueryClassifier(model_name_or_path="shahrukhx01/question-vs-statement-classifier")

for query in queries:
    result = question_classifier.run(query=query)
    if result[1] == "output_1":
        category = "question"
    else:
        category = "statement"

    print(f"Query: {query}, raw_output: {result}, class: {category}")

Query: Lord Eddard was the father of Arya Stark., raw_output: ({'query': 'Lord Eddard was the father of Arya Stark.'}, 'output_2'), class: statement
Query: Jon Snow was filmed in United Kingdom., raw_output: ({'query': 'Jon Snow was filmed in United Kingdom.'}, 'output_2'), class: statement
Query: who is the father of arya stark?, raw_output: ({'query': 'who is the father of arya stark?'}, 'output_1'), class: question
Query: Which country was jon snow filmed in?, raw_output: ({'query': 'Which country was jon snow filmed in?'}, 'output_1'), class: question


## Conclusion

The query classifier gives you more possibility to be more creative with the pipelines and use different retrieval nodes in a flexible fashion. Moreover, as in the case of Question vs Statement classifier you can also choose the queries which you want to send to the reader.

Finally, you also have the possible of bringing your own classifier and plugging it into either `TransformersQueryClassifier(model_name_or_path="<huggingface_model_name_or_file_path>")` or using the `SklearnQueryClassifier(model_name_or_path="url_to_classifier_or_file_path_as_pickle", vectorizer_name_or_path="url_to_vectorizer_or_file_path_as_pickle")`

## About us

This [Haystack](https://github.com/deepset-ai/haystack/) notebook was made with love by [deepset](https://deepset.ai/) in Berlin, Germany

We bring NLP to the industry via open source!  
Our focus: Industry specific language models & large scale QA systems.
  
Some of our other work: 
- [German BERT](https://deepset.ai/german-bert)
- [GermanQuAD and GermanDPR](https://deepset.ai/germanquad)
- [FARM](https://github.com/deepset-ai/FARM)

Get in touch:
[Twitter](https://twitter.com/deepset_ai) | [LinkedIn](https://www.linkedin.com/company/deepset-ai/) | [Slack](https://haystack.deepset.ai/community/join) | [GitHub Discussions](https://github.com/deepset-ai/haystack/discussions) | [Website](https://deepset.ai)

By the way: [we're hiring!](https://www.deepset.ai/jobs) 